In [2]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import os

In [12]:
def build_network_graph(network_df):
    """
    Build a directed graph from the Parquet-based network DataFrame.
    Nodes = LINKNO, Edges = (LINKNO -> DSLINKNO)
    """
    G = nx.DiGraph()
    for idx, row in network_df.iterrows():
        linkno = row['LINKNO']
        dslinkno = row['DSLINKNO']
        if pd.notnull(linkno):
            G.add_node(linkno)
        if pd.notnull(linkno) and pd.notnull(dslinkno):
            G.add_edge(linkno, dslinkno)
    return G

def find_upstream_linknos(G, linkno):
    """
    Given a directed graph G and a linkno, return all upstream LINKNOs (including itself).
    If linkno not in G, return an empty set.
    """
    if linkno not in G:
        return set()
    upstream = nx.ancestors(G, linkno)
    upstream.add(linkno)  # Include the gauge's own link
    return upstream

def process_vpu_catchments(
    vpu_code, 
    network_df, 
    gauge_df, 
    spatialite_dir,
    output_dir,
    dissolve=True
):
    """
    For a given VPU code:
      1) Build the connectivity graph from the master network DataFrame.
      2) Filter gauge_df for downstream gauges belonging to this VPU.
      3) For each downstream gauge, find all upstream LINKNOs.
      4) Load the relevant .spatialite file (one file per VPU) and layer.
      5) Select polygons matching the upstream LINKNOs and optionally dissolve them.
      6) Write to a shapefile (one shapefile per VPU).
    
    :param vpu_code: The VPU code to process (e.g., '101').
    :param network_df: Pandas DataFrame with columns [LINKNO, DSLINKNO, VPUCode].
    :param gauge_df: Pandas DataFrame with columns [LINKNO_for_watershed_Area_figure, VPUCode, DownstreamFlag].
    :param spatialite_dir: Directory where catchments_{vpu}.spatialite files are stored.
    :param output_dir: Directory to store output shapefiles.
    :param dissolve: Whether to dissolve polygons for each downstream gauge. Default = True.
    """
    # --- 1) Build the connectivity graph for this VPU ---
    network_subset = network_df[network_df['VPUCode'] == vpu_code].copy()
    G = build_network_graph(network_subset)
    
    # --- 2) Filter gauge_df for downstream gauges in this VPU ---
    downstream_gauges = gauge_df[
        (gauge_df['VPUCode'] == vpu_code) &
        (gauge_df['Upstream of'] == "downstream")
    ].copy()
    
    if downstream_gauges.empty:
        print(f"No downstream gauges found for VPU {vpu_code}. Skipping.")
        return
    
    print(f"Processing VPU {vpu_code}: {len(downstream_gauges)} downstream gauge(s).")

    # --- 3) For each downstream gauge, find all upstream LINKNOs ---
    gauge_upstream_dict = {}
    for idx, row in downstream_gauges.iterrows():
        gauge_id = row['LINKNO_for_watershed_Area_figure']
        if pd.isnull(gauge_id):
            continue
        upstream_set = find_upstream_linknos(G, gauge_id)
        gauge_upstream_dict[gauge_id] = upstream_set
    
    # --- 4) Load the relevant .spatialite file and layer ---
    # We assume the file is named catchments_{vpu_code}.spatialite
    # and the layer name is also catchments_{vpu_code}.
    vpu_str = str(int(float(vpu_code)))
    spatialite_path = os.path.join(spatialite_dir, f"catchments_{vpu_str}.spatialite")
    if not os.path.exists(spatialite_path):
        print(f"Spatialite file not found: {spatialite_path}. Skipping VPU {vpu_str}.")
        return
    
    layer_name = f"catchments_{vpu_str}"
    print(f"  Reading catchments from {spatialite_path}, layer='{layer_name}' ...")
    try:
        catchments_gdf = gpd.read_file(spatialite_path, driver='SQLite', layer=layer_name)
    except Exception as e:
        print(f"  Could not read layer '{layer_name}' from {spatialite_path}: {e}")
        return
    
    linkno_col = 'linkno'  # Adjust if your column is named differently
    if linkno_col not in catchments_gdf.columns:
        print(f"  Column '{linkno_col}' not found in {spatialite_path}. Check naming.")
        return
    
    # --- 5) Select polygons for upstream LINKNOs and optionally dissolve ---
    result_records = []
    
    for gauge_id, upstream_set in gauge_upstream_dict.items():
        if not upstream_set:
            continue
        
        subset_gdf = catchments_gdf[catchments_gdf[linkno_col].isin(upstream_set)]
        
        if subset_gdf.empty:
            continue
        
        if dissolve:
            # Dissolve all polygons into one boundary
            dissolved_geom = subset_gdf.dissolve()  # merges everything into one row
            dissolved_geom['gauge_id'] = gauge_id
            for idx2, row2 in dissolved_geom.iterrows():
                result_records.append({
                    'gauge_id': gauge_id,
                    'geometry': row2.geometry
                })
        else:
            # Keep polygons undissolved
            subset_gdf_copy = subset_gdf.copy()
            subset_gdf_copy['gauge_id'] = gauge_id
            for idx2, row2 in subset_gdf_copy.iterrows():
                result_records.append({
                    'gauge_id': gauge_id,
                    'geometry': row2.geometry
                })
    
    if not result_records:
        print(f"  No polygons found to save for VPU {vpu_code}.")
        return
    
    result_gdf = gpd.GeoDataFrame(result_records, geometry='geometry', crs=catchments_gdf.crs)
    
    # --- 6) Write to a shapefile (one shapefile per VPU) ---
    out_name = f"catchments_{vpu_code}_{'dissolved' if dissolve else 'undissolved'}.shp"
    out_path = os.path.join(output_dir, out_name)
    result_gdf.to_file(out_path, driver='ESRI Shapefile')
    print(f"  --> Wrote {len(result_gdf)} features to {out_path}.\n")


# -----------------------------
# Example usage:
# -----------------------------
if __name__ == "__main__":
    # Example input data paths
    network_parquet_path = '/Users/yubinbaaniya/Documents/GAUGE REVIEW/other calculation_ on master file/v2-master-table.parquet'
    gauge_excel_path = '/Users/yubinbaaniya/Documents/GAUGE REVIEW/other calculation_ on master file/master_file_with_most downstream_most_updated.xlsx'
    spatialite_folder = '/Users/yubinbaaniya/Downloads'  # e.g., catchments_101.spatialite
    output_folder = '/Users/yubinbaaniya/Documents/GAUGE REVIEW/other calculation_ on master file/catchment area'
    
    # Read input data
    network_df = pd.read_parquet(network_parquet_path)
    gauge_df = pd.read_excel(gauge_excel_path)
    
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Identify unique VPUs that have "downstream" gauges
    vpus_with_downstream = gauge_df.loc[gauge_df['Upstream of'] == "downstream", 'VPUCode'].dropna().unique()
    
    for vpu_code in vpus_with_downstream:
        # Process once with dissolve
        process_vpu_catchments(
            vpu_code=vpu_code,
            network_df=network_df,
            gauge_df=gauge_df,
            spatialite_dir=spatialite_folder,
            output_dir=output_folder,
            dissolve=True
        )
        
        # # Process once without dissolve
        # process_vpu_catchments(
        #     vpu_code=vpu_code,
        #     network_df=network_df,
        #     gauge_df=gauge_df,
        #     spatialite_dir=spatialite_folder,
        #     output_dir=output_folder,
        #     dissolve=False
        # )


Processing VPU 605.0: 5 downstream gauge(s).
  Reading catchments from /Users/yubinbaaniya/Downloads/catchments_605.spatialite, layer='catchments_605' ...
  --> Wrote 5 features to /Users/yubinbaaniya/Documents/GAUGE REVIEW/other calculation_ on master file/catchment area/catchments_605.0_dissolved.shp.

Processing VPU 604.0: 2 downstream gauge(s).
Spatialite file not found: /Users/yubinbaaniya/Downloads/catchments_604.spatialite. Skipping VPU 604.
Processing VPU 606.0: 2 downstream gauge(s).
Spatialite file not found: /Users/yubinbaaniya/Downloads/catchments_606.spatialite. Skipping VPU 606.
Processing VPU 602.0: 34 downstream gauge(s).
Spatialite file not found: /Users/yubinbaaniya/Downloads/catchments_602.spatialite. Skipping VPU 602.
Processing VPU 608.0: 56 downstream gauge(s).
Spatialite file not found: /Users/yubinbaaniya/Downloads/catchments_608.spatialite. Skipping VPU 608.
Processing VPU 607.0: 67 downstream gauge(s).
Spatialite file not found: /Users/yubinbaaniya/Downloads/c